In [1]:
import plotly.express as px
import pandas as pd
import pickle

In [109]:
df = pickle.load(open("data/loan_sum_by_state.pkd", "rb"))

In [118]:
unem = pd.read_csv(open("static/unemployment_county_14mo.txt"), sep="|", header=None)
unem.isna().sum()

0    0
1    5
2    5
3    5
4    5
5    5
6    5
7    5
8    5
dtype: int64

In [119]:
unem = unem.dropna()

In [120]:
indx = ['laus', 'fips_state', 'fips_count', 'title', 'period', 'labor', 'employed', 'unem_level', 'unem_rate']
idx = range(0,9)
zipdex = [(i[0], i[1]) for i in list(zip(idx, indx))]

d = dict()
for i in zipdex:
    d[i[0]] = i[1]

In [130]:
d

{0: 'laus',
 1: 'fips_state',
 2: 'fips_count',
 3: 'title',
 4: 'period',
 5: 'labor',
 6: 'employed',
 7: 'unem_level',
 8: 'unem_rate'}

In [121]:
unem = unem.rename(columns = d)

In [156]:
unem['period'] = unem.apply(lambda row: row['period'].strip(), axis=1)

In [177]:
unem = unem.drop(columns="state")

In [209]:
# unem['state'] = unem.apply(lambda row: str(row['title'].strip())[-2:], axis=1)

In [216]:
# unem['county'] = unem.apply(lambda row: str(row['title'].strip())[0:-10].strip(), axis=1)

In [241]:
unem['county'] = unem['title'].str.split(pat=',', expand=True)[0].str.split(pat=' ', expand=True)[1]

In [305]:
unem['state'] = unem['title'].str.split(pat=',', expand=True)[1]

KeyError: 'title'

In [244]:
pickle.dump(unem, open("data/unemployment_14mo_clean.pkd", "wb"))

In [359]:
unem = pickle.load(open("data/unemployment_14mo_clean.pkd", "rb"))

In [360]:
unem = unem[unem['period'] == 'Apr-20']

In [361]:
unem = unem.drop(columns=['laus', 'fips_state', 'fips_count', 'title']).reset_index().drop(columns='index')

In [364]:
unem.head()

,period,labor,employed,unem_level,unem_rate,state,county
0,Apr-20,"25,140","22,262","2,878",11.4,AL,Autauga
1,Apr-20,"93,597","79,140","14,457",15.4,AL,Baldwin
2,Apr-20,"8,116","7,310",806,9.9,AL,Barbour
3,Apr-20,"9,023","7,428","1,595",17.7,AL,Bibb
4,Apr-20,"23,944","21,685","2,259",9.4,AL,Blount


In [362]:
cols = ['labor', 'employed', 'unem_level']
for col in cols:
    unem[col] = unem.apply(lambda row: int(str(row[col]).strip().replace(",","")), axis=1)

ValueError: invalid literal for int() with base 10: '-'

In [363]:
unem['unem_rate'] = unem.apply(lambda row: float(str(row['unem_rate']).strip()), axis=1)

ValueError: could not convert string to float: '-'

In [296]:
pickle.dump(unem, open("data/unemployment_may20.pkd", "wb"))

In [292]:
unem.head()

,period,labor,employed,unem_level,unem_rate,state,county
0,May-20,25586,23593,1993,7.8,AL,Autauga
1,May-20,96285,87248,9037,9.4,AL,Baldwin
2,May-20,8700,7947,753,8.7,AL,Barbour
3,May-20,8814,7872,942,10.7,AL,Bibb
4,May-20,24584,23016,1568,6.4,AL,Blount


In [294]:
state_rate = unem.drop(columns=['period', 'county', 'unem_rate'])

In [322]:
state_rate['state'] = state_rate.apply(lambda row: str(row['state']).strip(), axis=1)

In [325]:
state_sums = state_rate.groupby(by=['state']).sum()

In [334]:
state_sums = state_sums.drop(labels=['None', 'PR'])

KeyError: "['None' 'PR'] not found in axis"

In [337]:
state_sums['rate'] = state_sums.apply(lambda row: row['unem_level']/row['labor'], axis=1)

In [351]:
state_sums = state_sums.sort_values(by='rate')

In [356]:
pickle.dump(state_sums, open("data/unemployment_rates_by_state_may20.pkd", "wb"))

In [352]:
fig = px.bar(state_sums, x=state_sums.index, y="rate",
                  title="May Unemployment")

In [354]:
fig.update_xaxes(title_text='State')
fig.update_yaxes(title_text='Unemployment (% of Population)')
fig.show()

In [357]:
fig.write_html("templates/may_unemployment.html", include_plotlyjs="cdn")

In [358]:
fig.write_image("static/may_unemployment.png")

### fips

In [92]:
def make_5_digit_fips():
    df.apply(lambda row: (str(0) + str(int(row['fips_state'])))[-2:] + (str(00) + str(int(row['fips_count'])))[-3:], axis=1)

In [94]:
make_5_digit_fips()

In [90]:
df = df.drop(columns="fips")

In [59]:
a = '00003'

In [60]:
a[-2:]

'03'

In [56]:
unem

,laus,fips_state,fips_count,title,period,labor,employed,unem_level,unem_rate,fips
0,CN0100100000000,1.0,1.0,"Autauga County, AL",May-19,"26,271","25,680",591,2.2,01
1,CN0100300000000,1.0,3.0,"Baldwin County, AL",May-19,"98,311","96,129","2,182",2.2,01
2,CN0100500000000,1.0,5.0,"Barbour County, AL",May-19,"8,416","8,152",264,3.1,01
3,CN0100700000000,1.0,7.0,"Bibb County, AL",May-19,"8,605","8,391",214,2.5,01
4,CN0100900000000,1.0,9.0,"Blount County, AL",May-19,"25,136","24,589",547,2.2,01
...,...,...,...,...,...,...,...,...,...,...
45061,CN7214500000000,72.0,145.0,"Vega Baja Municipio, PR",Jun-20(p),12967,11408,1559,12,072
45062,CN7214700000000,72.0,147.0,"Vieques Municipio, PR",Jun-20(p),2603,2375,228,8.8,072
45063,CN7214900000000,72.0,149.0,"Villalba Municipio, PR",Jun-20(p),6757,6159,598,8.9,072
45064,CN7215100000000,72.0,151.0,"Yabucoa Municipio, PR",Jun-20(p),8164,7395,769,9.4,072


In [58]:
unem[unem['fips_state'] == 51]

,laus,fips_state,fips_count,title,period,labor,employed,unem_level,unem_rate,fips
2819,CN5100100000000,51.0,1.0,"Accomack County, VA",May-19,"16,601","16,079",522,3.1,051
2820,CN5100300000000,51.0,3.0,"Albemarle County, VA",May-19,"58,002","56,554","1,448",2.5,051
2821,CN5100500000000,51.0,5.0,"Alleghany County, VA",May-19,"6,918","6,708",210,3.0,051
2822,CN5100700000000,51.0,7.0,"Amelia County, VA",May-19,"6,265","6,077",188,3.0,051
2823,CN5100900000000,51.0,9.0,"Amherst County, VA",May-19,"15,054","14,586",468,3.1,051
...,...,...,...,...,...,...,...,...,...,...
44794,CN5180000000000,51.0,800.0,"Suffolk city, VA",Jun-20(p),"44,364","40,465","3,899",8.8,051
44795,CN5181000000000,51.0,810.0,"Virginia Beach city, VA",Jun-20(p),"232,335","212,583","19,752",8.5,051
44796,CN5182000000000,51.0,820.0,"Waynesboro city, VA",Jun-20(p),"11,201","10,216",985,8.8,051
44797,CN5183000000000,51.0,830.0,"Williamsburg city, VA",Jun-20(p),"6,820","6,032",788,11.6,051
